In [ ]:
!which chromedriver

In [1]:
import pymongo
from bs4 import BeautifulSoup as bs
import os
import requests
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', executable_path, headless=False)
import pandas as pd
from splinter import Browser

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", usr/local/bin/chromedriver, headless=False)

### NASA Mars News
* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

```python
# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."
```


In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.NASA_Mars_news_db
collection = db.items

In [5]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [78]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
#results = soup.find_all('a')
results = soup.find_all('a', href=True)
#results = soup.find_all('a', href=True).get_text() - illegal
#results = soup.find_all('a').get('href') - illegal
#results = soup.find_all(href=True)
#results = soup.find_all(href=True, link=False) - legal, but had no effect
#results = soup.find_all('a', href=True, img alt=False) - illegal (img alone wasn't, but had no effect)
#results = soup.find_all('a', href=True, news=True) - illegal
#results = soup.find_all('a', news=True) - nothing
#results = soup.find_all(news=True) - nothing
#results = soup.find_all('news'=True) - illegal
results

[<a class="nasa_logo" href="http://www.nasa.gov" target="_blank" title="visit nasa.gov">NASA</a>,
 <a class="top_logo" href="https://science.nasa.gov/" target="_blank" title="Explore NASA Science">NASA Science</a>,
 <a class="sub_logo" href="/mars-exploration/#" title="Mars">Mars Exploration Program</a>,
 <a class="visuallyhidden focusable" href="#page">Skip Navigation</a>,
 <a class="menu_button" href="javascript:void(0);" id="menu_button">
 <span class="menu_icon">
 menu
 </span>
 </a>,
 <a href="/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/">
 <div class="rollover_description">
 <div class="rollover_description_inner">
 Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.
 </div>
 <div class="overlay_arrow">
 <img alt="More" src="/assets/overlay-arrow.png"/>
 </div>
 </div>
 <img alt="NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Ac

In [65]:
# Loop through returned results
for result in results:
    # Error handling
    try:

        #title = soup.href - nothing
        # Identify and return title of listing
        #title = result.find(class='rollover_description_inner').text
        # Identify and return price of listing
        #price = result.a.span.text
        # Identify and return link to listing
        #link = result.a['href']

        # Run only if title, price, and link are available
        #if (title and price and link):
            # Print results
        print('-------------')
        print(title)
        #    print(price)
        #    print(link)

            # Dictionary to be inserted as a MongoDB document
        post = {
                'title': title,
         #       'price': price,
          #      'url': link
            }

        collection.insert_one(post)

    except Exception as e:
        print(e)

-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None
-------------
None


### JPL Mars Space Images - Featured Image
* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

```python
# Example:
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
```


### Mars Facts
* Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table.


In [ ]:
url = 'https://space-facts.com/mars/'

In [ ]:
tables = pd.read_html(url)
type(tables)

In [ ]:
tables[0]

In [ ]:
mars_facts_df = tables[0]

mars_facts_df.set_index([0], inplace=True)

mars_facts_df

In [ ]:
mars_facts_df.rename(columns=mars_facts_df.iloc[0]).drop(mars_facts_df.index[0])
mars_facts_df

In [ ]:
mars_facts_html_table = mars_facts_df.to_html()
mars_facts_html_table = mars_facts_html_table.replace('\n', '')
mars_facts_html_table
print(mars_facts_html_table)

### Mars Hemispheres
* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mars's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save the url string for the full resolution hemisphere image as well as the Hemisphere title containing the hemisphere name. Use a Python dictionary to store these values using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]


In [ ]:
#These are not actually FULL resolution, but those are enormous tif downloads

#https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
#https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
#https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
#https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg

In [ ]:
#hemisphere_image_urls = [
#    {"title": "Valles Marineris Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg"},
#    {"title": "Cerberus Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"},
#    {"title": "Schiaparelli Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg"},
#    {"title": "Syrtis Major Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg"},
#]


## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will call your `scrape_mars.scrape` function. Note that you'll have to import `scrape_mars.py`. 

  * Store the dictionary that gets returned to your MongoDB.

* Create an index route `/` that will query your Mongo database and pass the Mars data into an HTML template to be displayed. 

* Create a template HTML file called `index.html` that will take the dictionary of Mars data and display its values in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

![final_app_part1.png](Images/final_app_part1.png)
![final_app_part2.png](Images/final_app_part2.png)